**imports**

In [ ]:
import torch
import torch.nn as nn
import torch_neuronx
import os
from diffusers import StableDiffusionPipeline, DPMSolverMultistepScheduler
from diffusers.models.unet_2d_condition import UNet2DConditionOutput
from matplotlib import pyplot as plt
from matplotlib import image as mpimg
import time
from IPython.display import clear_output
from diffusers.models.cross_attention import CrossAttention
import numpy as np

clear_output(wait=False)

**If this is the first time you trace a model, it will need to be compiled and optimized for Inf2 through neuron. This step is only required once. After the model is compiled, it is loaded onto the NeuronCores and a single prompt is given** 

In [ ]:

class UNetWrap(nn.Module):
    def __init__(self, unet):
        super().__init__()
        self.unet = unet

    def forward(self, sample, timestep, encoder_hidden_states, cross_attention_kwargs=None):
        out_tuple = self.unet(sample, timestep, encoder_hidden_states, return_dict=False)
        return out_tuple

class NeuronUNet(nn.Module):
    def __init__(self, unetwrap):
        super().__init__()
        self.unetwrap = unetwrap
        self.config = unetwrap.unet.config
        self.in_channels = unetwrap.unet.in_channels
        self.device = unetwrap.unet.device

    def forward(self, sample, timestep, encoder_hidden_states, cross_attention_kwargs=None):
        sample = self.unetwrap(sample, timestep.float().expand((sample.shape[0],)), encoder_hidden_states)[0]
        return UNet2DConditionOutput(sample=sample)

def get_attention_scores(self, query, key, attn_mask):       
    dtype = query.dtype

    if self.upcast_attention:
        query = query.float()
        key = key.float()

    if(query.size() == key.size()):
        attention_scores = cust_badbmm(
            key,
            query.transpose(-1, -2)
        )

        if self.upcast_softmax:
            attention_scores = attention_scores.float()

        attention_probs = torch.nn.functional.softmax(attention_scores, dim=1).permute(0,2,1)
        attention_probs = attention_probs.to(dtype)

    else:
        attention_scores = cust_badbmm(
            query,
            key.transpose(-1, -2)
        )

        if self.upcast_softmax:
            attention_scores = attention_scores.float()

        attention_probs = torch.nn.functional.softmax(attention_scores, dim=-1)
        attention_probs = attention_probs.to(dtype)
        
    return attention_probs

def cust_badbmm(a, b):
    bmm = torch.bmm(a, b)
    scaled = bmm * 0.125
    return scaled


# For saving compiler artifacts
COMPILER_WORKDIR_ROOT = 'sd2_compile_dir'

model_id = "stabilityai/stable-diffusion-2-1-base"
pipe = StableDiffusionPipeline.from_pretrained(model_id, torch_dtype=torch.float32)
pipe.scheduler = DPMSolverMultistepScheduler.from_config(pipe.scheduler.config)

# Replace original cross-attention module with custom cross-attention module for better performance
CrossAttention.get_attention_scores = get_attention_scores

pipe.unet = NeuronUNet(UNetWrap(pipe.unet))

# Compile unet - FP32
sample_1b = torch.randn([1, 4, 64, 64])
timestep_1b = torch.tensor(999).float().expand((1,))
encoder_hidden_states_1b = torch.randn([1, 77, 1024])
example_inputs = sample_1b, timestep_1b, encoder_hidden_states_1b

pipe.unet.unetwrap = torch_neuronx.trace(
    pipe.unet.unetwrap,
    example_inputs,
    compiler_workdir=os.path.join(COMPILER_WORKDIR_ROOT, 'unet'),
    compiler_args=["--model-type=unet-inference"]
)

# save compiled unet
unet_filename = os.path.join(COMPILER_WORKDIR_ROOT, 'unet/model.pt')
torch.jit.save(pipe.unet.unetwrap, unet_filename)

# load previously compiled unet
# unet_filename = os.path.join(COMPILER_WORKDIR_ROOT, 'unet/model.pt')
# pipe.unet.unetwrap = torch.jit.load(unet_filename)

# Load the compiled UNet onto two neuron cores.
device_ids = [0,1]
pipe.unet.unetwrap = torch_neuronx.DataParallel(pipe.unet.unetwrap, device_ids, set_dynamic_batching=False)

# # Compile vae post_quant_conv
post_quant_conv_in = torch.randn([1, 4, 64, 64])
pipe.vae.post_quant_conv = torch_neuronx.trace(
    pipe.vae.post_quant_conv, 
    post_quant_conv_in,
    compiler_workdir=os.path.join(COMPILER_WORKDIR_ROOT, 'vae_post_quant_conv'),
)

# # Save the compiled vae post_quant_conv
post_quant_conv_filename = os.path.join(COMPILER_WORKDIR_ROOT, 'vae_post_quant_conv/model.pt')
torch.jit.save(pipe.vae.post_quant_conv, post_quant_conv_filename)

# Load the previously-compiled vae post_quant_conv
# post_quant_conv_filename = os.path.join(COMPILER_WORKDIR_ROOT, 'vae_post_quant_conv/model_cast_fp32.pt')
# pipe.vae.post_quant_conv = torch.jit.load(post_quant_conv_filename)

# Compile vae decoder
decoder_in = torch.randn([1, 4, 64, 64])
pipe.vae.decoder = torch_neuronx.trace(
    pipe.vae.decoder, 
    decoder_in, 
    compiler_workdir=os.path.join(COMPILER_WORKDIR_ROOT, 'vae_decoder'),
)

# Save the compiled vae decoder
decoder_filename = os.path.join(COMPILER_WORKDIR_ROOT, 'vae_decoder/model.pt')
torch.jit.save(pipe.vae.decoder, decoder_filename)

# # Load the previously-compiled vae decoder
# decoder_filename = os.path.join(COMPILER_WORKDIR_ROOT, 'vae_decoder/model.pt')
# pipe.vae.decoder = torch.jit.load(decoder_filename)

prompt = "a photo of an astronaut riding a horse on mars"
image = pipe(prompt).images[0]
image.save("image.png")

plt.title("Image")
plt.xlabel("X pixel scaling")
plt.ylabel("Y pixels scaling")

image = mpimg.imread("image.png")
plt.imshow(image)
plt.show()


**Now that the model is compiled, you can reload it with any number of prompts. Edit the Prompts below to see what you can create.**

In [ ]:

# load previously compiled unet
unet_filename = os.path.join(COMPILER_WORKDIR_ROOT, 'unet/model.pt')
pipe.unet.unetwrap = torch.jit.load(unet_filename)

# Load the compiled UNet onto two neuron cores.
device_ids = [0,1]
pipe.unet.unetwrap = torch_neuronx.DataParallel(pipe.unet.unetwrap, device_ids, set_dynamic_batching=False)

# Load the previously-compiled vae post_quant_conv
post_quant_conv_filename = os.path.join(COMPILER_WORKDIR_ROOT, 'vae_post_quant_conv/model.pt')
pipe.vae.post_quant_conv = torch.jit.load(post_quant_conv_filename)

 # Load the previously-compiled vae decoder
decoder_filename = os.path.join(COMPILER_WORKDIR_ROOT, 'vae_decoder/model.pt')
pipe.vae.decoder = torch.jit.load(decoder_filename)

prompt = ["a photo of an astronaut riding a horse on mars",
          "sonic on the moon",
          "elvis playing guitar while eating a hotdog",
          "saved by the bell",
          "engineers eating lunch at the opera",
          "panda eating bamboo on a plane",
          "A digital illustration of a steampunk flying machine in the sky with cogs and mechanisms, 4k, detailed, trending in artstation, fantasy vivid colors",
          "kids playing soccer at the FIFA World Cup"
         ]

plt.title("Image")
plt.xlabel("X pixel scaling")
plt.ylabel("Y pixels scaling")

total_time = 0
for x in prompt:
    start_time = time.time()
    image = pipe(x).images[0]
    total_time = total_time + (time.time()-start_time)
    image.save("image.png")
    image = mpimg.imread("image.png")
    #clear_output(wait=True)
    plt.imshow(image)
    plt.show()
print("Average time: ", np.round((total_time/len(prompt)), 2), "seconds")



 



**Now have Fun**

In [ ]:
user_input = ""
print("Enter Prompt, type exit to quit")
while user_input != "exit": 
    total_time = 0
    user_input = input("What prompt would you like to give?  ")
    if user_input == "exit":
        break
    start_time = time.time()
    image = pipe(user_input).images[0]
    total_time = total_time + (time.time()-start_time)
    image.save("image.png")

    plt.title("Image")
    plt.xlabel("X pixel scaling")
    plt.ylabel("Y pixels scaling")

    image = mpimg.imread("image.png")
    plt.imshow(image)
    plt.show()
    print("time: ", np.round(total_time, 2), "seconds")
    
